# Лабораторная работа №2 Разработка синтаксического LL(1)-анализатора


In [1]:
from collections import deque
import pandas as pd

In [2]:
dict_analyzer = {}

In [5]:
df = pd.read_excel("ll1.xlsx", index_col = "№")
df.head()

,множество направляющих символов,next,return,stack,accept,error
№,,,,,,
1,"not true false const perem ) , ( name end",22,0,0,0,1
2,and,24,0,0,0,0
3,or,27,0,0,0,0
4,") , end",30,0,0,0,1
5,not,31,0,0,0,0


In [6]:
for i in range(1, df.shape[0]+1):
  dict_analyzer[i] = []
  for j in df.columns:
    dict_analyzer[i].append(df[j][i])

dict_analyzer

{1: ['not true false const perem ) , ( name end ', 22, 0, 0, 0, 1],
 2: ['and ', 24, 0, 0, 0, 0],
 3: ['or', 27, 0, 0, 0, 0],
 4: [') , end', 30, 0, 0, 0, 1],
 5: ['not', 31, 0, 0, 0, 0],
 6: ['true', 33, 0, 0, 0, 0],
 7: ['false', 34, 0, 0, 0, 0],
 8: ['const perem ( name ', 35, 0, 0, 0, 1],
 9: ['+', 37, 0, 0, 0, 0],
 10: ['-', 40, 0, 0, 0, 0],
 11: ['and or ) , end', 43, 0, 0, 0, 1],
 12: ['const perem ( name ', 44, 0, 0, 0, 1],
 13: ['* ', 46, 0, 0, 0, 0],
 14: ['/', 49, 0, 0, 0, 0],
 15: ['+ - or and ) , end', 52, 0, 0, 0, 1],
 16: ['const ', 53, 0, 0, 0, 0],
 17: ['perem ', 54, 0, 0, 0, 0],
 18: ['( ', 55, 0, 0, 0, 0],
 19: ['name', 58, 0, 0, 0, 1],
 20: [', ', 63, 0, 0, 0, 0],
 21: [') ', 66, 0, 0, 0, 1],
 22: ['not true false const perem ) , ( name end ', 5, 0, 1, 0, 1],
 23: ['and or ) , end', 2, 0, 0, 0, 1],
 24: ['and', 25, 0, 0, 1, 1],
 25: ['not true false const perem ) , ( name end ', 5, 0, 1, 0, 1],
 26: ['and or ) , end', 2, 0, 0, 0, 1],
 27: ['or ', 28, 0, 0, 1, 1],
 2

In [18]:
def ll1_analyzer(word):
  n = 1
  word = word.split()
  word.append("end")
  stask_memory = deque()

  for i in range(len(word)):
    
    letter = word[i]
    print("Letter = ", letter)

    is_accept = False
    is_error = False
    is_end_letter = False

    while not(is_accept) or not(is_error) or not(is_end_letter) :
      print(n, " -> ")

      is_end_letter = (letter == "end" and (letter in dict_analyzer[n][0]) and len(stask_memory) == 0)

      letter = ("const" if letter.isdigit() else letter)

      m = ", ) ( true false * / + - or and not end const name".split()
      if i != len(word) - 1 and (letter not in m):
        letter = ("name" if ((word[i + 1] == '(')) else letter)
      
      
      letter = ("perem" if letter not in m else letter)
      
      if ((letter in dict_analyzer[n][0]) and not(is_end_letter)):

        # Return
        new_n = n
        if dict_analyzer[n][2] ==  1:
          print(n, "From stack")
          new_n = stask_memory.pop()
          print(stask_memory)
        else:
          new_n = dict_analyzer[n][1]

        # Stack
        if dict_analyzer[n][3] ==  1:
          print(n, "To stack")
          stask_memory.append(n+1)
          print(stask_memory)

        # Accept
        if dict_analyzer[n][4] ==  1:
          is_accept = True
          print(n, "Accept")
          n = new_n
          break

        n = new_n

      elif is_end_letter:
        break
      
      # Error
      elif dict_analyzer[n][5] ==  0:
          n += 1
      else: 
        is_error = True
        break
    
    if is_error: 
      print("Error!")
      break

    if is_end_letter:
      print("This word belongs to this grammar")
      break

При вводе слова для проверки все символы необходимо разделять пробелом.

In [19]:
ll1_analyzer("( 2 + 3 ) * 14")

Letter =  (
1  -> 
22  -> 
22 To stack
deque([23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 36, 45])
16  -> 
17  -> 
18  -> 
55  -> 
55 Accept
Letter =  2
56  -> 
56 To stack
deque([23, 36, 45, 57])
1  -> 
22  -> 
22 To stack
deque([23, 36, 45, 57, 23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 36, 45, 57, 23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 36, 45, 57, 23, 36, 45])
16  -> 
53  -> 
53 From stack
deque([23, 36, 45, 57, 23, 36])
53 Accept
Letter =  +
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23, 36, 45, 57, 23])
36  -> 
9  -> 
37  -> 
37 Accept
Letter =  3
38  -> 
38 To stack
deque([23, 36, 45, 57, 23, 39])
12  -> 
44  -> 
44 To stack
deque([23, 36, 45, 57, 23, 39, 45])
16  -> 
53  -> 
53 From stack
deque([23, 36, 45, 57, 23, 39])
53 Accept
Letter =  )
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23, 36, 45, 57, 23])
39  -> 
9  -> 
10  -> 
11  -> 
43  -> 
43 From 

В данном формальном языке есть некоторое ограничение: логические операции можно применять только к true, false и функции.

In [20]:
ll1_analyzer("17 + xyz ( abc , x ) * ref * 52 and not true")

Letter =  17
1  -> 
22  -> 
22 To stack
deque([23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 36, 45])
16  -> 
53  -> 
53 From stack
deque([23, 36])
53 Accept
Letter =  +
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23])
36  -> 
9  -> 
37  -> 
37 Accept
Letter =  xyz
38  -> 
38 To stack
deque([23, 39])
12  -> 
44  -> 
44 To stack
deque([23, 39, 45])
16  -> 
17  -> 
18  -> 
19  -> 
58  -> 
58 Accept
Letter =  (
59  -> 
59 Accept
Letter =  abc
60  -> 
60 To stack
deque([23, 39, 45, 61])
1  -> 
22  -> 
22 To stack
deque([23, 39, 45, 61, 23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 39, 45, 61, 23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 39, 45, 61, 23, 36, 45])
16  -> 
17  -> 
54  -> 
54 From stack
deque([23, 39, 45, 61, 23, 36])
54 Accept
Letter =  ,
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23, 39, 45, 61, 23])
36  -> 
9  -> 
10  -> 
11  -> 
43  -> 
43 From stack
deque([

В данном языке мы не можем перемножать и складывать предикаты, поэтому ниже получим ошибку

In [21]:
ll1_analyzer("17 + xyz ( abc , x ) * ref * 52 + not true")

Letter =  17
1  -> 
22  -> 
22 To stack
deque([23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 36, 45])
16  -> 
53  -> 
53 From stack
deque([23, 36])
53 Accept
Letter =  +
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23])
36  -> 
9  -> 
37  -> 
37 Accept
Letter =  xyz
38  -> 
38 To stack
deque([23, 39])
12  -> 
44  -> 
44 To stack
deque([23, 39, 45])
16  -> 
17  -> 
18  -> 
19  -> 
58  -> 
58 Accept
Letter =  (
59  -> 
59 Accept
Letter =  abc
60  -> 
60 To stack
deque([23, 39, 45, 61])
1  -> 
22  -> 
22 To stack
deque([23, 39, 45, 61, 23])
5  -> 
6  -> 
7  -> 
8  -> 
35  -> 
35 To stack
deque([23, 39, 45, 61, 23, 36])
12  -> 
44  -> 
44 To stack
deque([23, 39, 45, 61, 23, 36, 45])
16  -> 
17  -> 
54  -> 
54 From stack
deque([23, 39, 45, 61, 23, 36])
54 Accept
Letter =  ,
45  -> 
13  -> 
14  -> 
15  -> 
52  -> 
52 From stack
deque([23, 39, 45, 61, 23])
36  -> 
9  -> 
10  -> 
11  -> 
43  -> 
43 From stack
deque([